# Setup Gym Environment

```py
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05 # when to give up
my_config["start_obs_capture"] = 0.05 # when to capture 
my_config["time_step_timeout_factor"] = 1.0 # how late is OK
my_config["act_buf_len"] = 3 # how many past actions
my_config["reset_act_buf"] = True # resect action buffer on reset
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2
```

This section needs to be setup for any method

In [1]:
debugAsGym = False
testResult = True

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 224
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 4,
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 3, # 1 is High Speed Ring, 2 is 0-400m, 
}

In [2]:
if debugAsGym:
    env = gymnasium.make("real-time-gym-v1", config=my_config)

In [3]:
if debugAsGym:
    env.reset()

# Register the environment in a way that RLlib is happy

In [4]:
if not debugAsGym and not testResult:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [5]:
if not debugAsGym and not testResult:
    import ray
    ray.shutdown()
    ray.init()

In [6]:
if not debugAsGym and not testResult:
    from ray.rllib.algorithms.ppo import PPOConfig

    algo = (
        PPOConfig()
        .resources(
            num_gpus=1
            )
        .rollouts(
            num_rollout_workers=1,
            enable_connectors=True,
            batch_mode="truncate_episodes",
            #rollout_fragment_length=256
            )
        .framework("torch")
        .environment(
            env="gt-rtgym-env-v1",
            disable_env_checking=True,
            render_env=False,
            )
        .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            train_batch_size=155,
            #num_sgd_iter=8,
            #clip_param=0.2,
            #model={"fcnet_hiddens": [1, 8]},
        )
        .build()
    )

In [7]:
if not debugAsGym and not testResult:
    N = 1000

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 10 == 0:
            print("Saved", n)
            algo.save()
            
    algo.save()


In [8]:
if not debugAsGym and not testResult:
    N = 1000

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 50 == 0:
            print("Saved", n)
            algo.save()
            
    algo.save()

In [9]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
#my_config["ep_max_length"] = 224
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 4,
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 3, # 1 is High Speed Ring, 2 is 0-400m, 
}

In [10]:
if not debugAsGym and testResult:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) 

In [11]:
if not debugAsGym and testResult:
    from ray.rllib.algorithms.algorithm import Algorithm
    algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-19_07-37-37z3d6v2w2/checkpoint_000061")
    #algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-19_07-37-37z3d6v2w2/checkpoint_002000")

2023-05-21 14:43:04,927	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-19_07-37-37z3d6v2w2/checkpoint_000061! Trying to extract checkpoint info from other files found in that dir.
2023-05-21 14:43:07,521	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(RolloutWorker pid=14836) GT Real Time instantiated
(RolloutWorker pid=14836) GT AI Server instantiated for rtgym
(RolloutWorker pid=14836) still simple reward system
(RolloutWorker pid=14836) starting up on localhost port 9999
(RolloutWorker pid=14836) Waiting for a connection
(RolloutWorker pid=14836) Connection from ('127.0.0.1', 60373)


2023-05-21 14:43:31,236	INFO trainable.py:172 -- Trainable.setup took 26.257 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [12]:
if not debugAsGym and testResult:
    result = algo.train()

(RolloutWorker pid=14836) reset triggered
(RolloutWorker pid=14836) reload save for track : 4


(RolloutWorker pid=14836) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 4.525002099999927
(RolloutWorker pid=14836)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=14836) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.25556329999991334
(RolloutWorker pid=14836)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=14836) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.1296612999999951
(RolloutWorker pid=14836)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


In [ ]:
if not debugAsGym and testResult:

    policy = algo.get_policy()
    #print(policy.model)
    model = policy.model
    print(model)